# Silver to Gold
### Overview
This notebook focuses on creating business level aggregate views. These are common questions business users may have and would like to visualize quickly.
Using the keywords `CREATE OR REFRESH MATERIALIZED VIEW` creates and manages the views.

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW product_sales
AS
SELECT 
    p.product_id,
    p.Name AS ProductName,
    p.list_price,
    COUNT(sod.sales_order_id) AS NumberOfOrders,
    SUM(sod.order_qty) AS TotalQuantitySold,
    SUM(sod.line_total) AS TotalRevenue
FROM 
    adventureworks.silver.product p
JOIN 
    adventureworks.silver.salesorderdetail sod ON p.product_id = sod.product_id
GROUP BY 
    p.product_id, p.Name, p.list_price
ORDER BY 
    TotalRevenue DESC
LIMIT 10;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW customer_purchase
AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS CustomerName,
    COUNT(soh.sales_order_id) AS NumberOfOrders,
    SUM(soh.total_due) AS TotalSpent
FROM 
    adventureworks.silver.customer c
JOIN 
    adventureworks.silver.salesorderheader soh ON c.customer_id = soh.customer_id
GROUP BY 
    c.customer_id, c.first_name, c.last_name
ORDER BY 
    TotalSpent DESC;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW customers_by_purchase
AS
WITH CustomerPurchases AS (
    SELECT
        c.customer_id,
        c.first_name + ' ' + c.last_name AS CustomerName,
        SUM(soh.total_due) AS TotalSpent
    FROM
        adventureworks.silver.Customer c
    JOIN
        adventureworks.silver.SalesOrderHeader soh ON c.customer_id = soh.customer_id
    GROUP BY
        c.customer_id, c.first_name, c.last_name
)
SELECT
    CASE
        WHEN TotalSpent >= 10000 THEN 'High Value'
        WHEN TotalSpent >= 5000 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS CustomerSegment,
    COUNT(*) AS NumberOfCustomers,
    AVG(TotalSpent) AS AverageSpend,
    SUM(TotalSpent) AS TotalRevenue
FROM
    CustomerPurchases
GROUP BY
    CASE
        WHEN TotalSpent >= 10000 THEN 'High Value'
        WHEN TotalSpent >= 5000 THEN 'Medium Value'
        ELSE 'Low Value'
    END
ORDER BY
    AVG(TotalSpent) DESC;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW sales_by_category
AS
SELECT 
    pc.Name AS CategoryName,
    COUNT(DISTINCT p.product_id) AS NumberOfProducts,
    SUM(sod.order_qty) AS TotalQuantitySold,
    SUM(sod.line_total) AS TotalRevenue,
    AVG(sod.line_total) AS AverageOrderValue
FROM 
    adventureworks.silver.ProductCategory pc
JOIN 
    adventureworks.silver.Product p ON pc.product_category_id = p.product_category_id
JOIN 
    adventureworks.silver.SalesOrderDetail sod ON p.product_id = sod.product_id
GROUP BY 
    pc.Name
ORDER BY 
    TotalRevenue DESC;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW monthly_sales
AS
SELECT
    YEAR(soh.order_date) AS OrderYear,
    MONTH(soh.order_date) AS OrderMonth,
    COUNT(DISTINCT soh.sales_order_id) AS NumberOfOrders,
    COUNT(DISTINCT soh.customer_id) AS NumberOfCustomers,
    SUM(soh.total_due) AS MonthlyRevenue,
    SUM(soh.total_due)/COUNT(DISTINCT soh.sales_order_id) AS AverageOrderValue
FROM
    adventureworks.silver.SalesOrderHeader soh
GROUP BY
    YEAR(soh.order_date),
    MONTH(soh.order_date)
ORDER BY
    OrderYear, OrderMonth;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW products_purchased_together
AS
SELECT
    p1.product_id AS Product1ID,
    p1.Name AS Product1Name,
    p2.product_id AS Product2ID,
    p2.Name AS Product2Name,
    COUNT(*) AS PurchasedTogetherCount
FROM
    adventureworks.silver.SalesOrderDetail sod1
JOIN
    adventureworks.silver.SalesOrderDetail sod2 ON sod1.sales_order_id = sod2.sales_order_id
    AND sod1.product_id < sod2.product_id
JOIN
    adventureworks.silver.Product p1 ON sod1.product_id = p1.product_id
JOIN
    adventureworks.silver.Product p2 ON sod2.product_id = p2.product_id
GROUP BY
    p1.product_id, p1.Name, p2.product_id, p2.Name
HAVING
    COUNT(*) > 5
ORDER BY
    PurchasedTogetherCount DESC;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW sales_by_region
AS
SELECT
    a.state_province,
    a.country_region,
    COUNT(DISTINCT soh.sales_order_id) AS NumberOfOrders,
    COUNT(DISTINCT c.customer_id) AS NumberOfCustomers,
    SUM(soh.total_due) AS TotalRevenue,
    SUM(soh.total_due)/COUNT(DISTINCT c.customer_id) AS AverageRevenuePerCustomer
FROM
    adventureworks.silver.Address a
JOIN
    adventureworks.silver.CustomerAddress ca ON a.address_id = ca.address_id
JOIN
    adventureworks.silver.Customer c ON ca.customer_id = c.customer_id
JOIN
    adventureworks.silver.SalesOrderHeader soh ON c.customer_id = soh.customer_id
GROUP BY
    a.state_province, a.country_region
ORDER BY
    TotalRevenue DESC;